In [ ]:
# Test scripts for the UR3e Arm
# Liam Campbell
# July 2024 @ Tufts CEEO

# Make sure ik_solver folder and MyUR3e.py are downloaded in JupyterHub Library

In [ ]:
import MyUR3e
import rclpy
import time
import numpy as np
from IPython.display import clear_output
from IPython.display import HTML

rclpy.init()

myrobot = MyUR3e.MyUR3e()

# UNCOMMENT THIS FOR DEBUGGING LOGS
#myrobot.get_logger().set_level(rclpy.logging.LoggingSeverity.DEBUG)

# Change these booleans to turn tests on / off
gripper_test = False
read_data = False
invalid_ik_test = False
wave_test = False
wrist_test = False
relative_test = False
line_test = False
circle_test = False

In [ ]:
# GRIPPER TEST
if gripper_test:
    myrobot.move_gripper(255,255,100) # close
    time.sleep(1)
    myrobot.move_gripper(0,255,100)
    
    print(myrobot.get_gripper())

In [ ]:
# MONITOR JOINT STATES TOPIC

if read_data:
    while True:
        clear_output(wait=True)
        coordinates = joint_states.get_global()
        for i in range(0,6):
            print(f"Name: {myrobot.get_joints()['name'][i]}")
            print(f"    Position: {myrobot.get_joints()['position'][i]}")
            print(f"    Velocity: {myrobot.get_joints()['velocity'][i]}")
            print(f"    Effort: {myrobot.get_joints()['effort'][i]}")
        print(f"Name: Wrench")
        print(f"    Force: {myrobot.get_force()['force']}")
        print(f"    Torque: {myrobot.get_force()['torque']}")
        time.sleep(0.05)

In [ ]:
# INVALID IK TEST

line = []
for i in range(10):
    line.append([0.375, -0.3+i*0.1,  0.258, 0,  i*30,  0])

try:
    myrobot.move_global(line,time_step=(5,1),sim=(not invalid_ik_test))
except Exception as e:
    print(e)

In [ ]:
# WAVE TEST

traj = []
for i in range(-60,60):
    point = [0.2, 0.3-(i/700),  0.3+0.05*np.sin(2*3.1415*(60+i)/120), 0,  i,  0]
    traj.append(point)

myrobot.move_global(traj,time_step=(5,0.1),sim=(not wave_test))

In [ ]:
# WRIST TEST
# SOME FUNKY STUFF GOING ON

traj = [[0.2,0.25,0.3,0,0,0],
       [0.2,0.25,0.3,0,45,0],
       [0.2,0.25,0.3,45,45,45],
       [0.2,0.25,0.3,45,45,90],
       [0.2,0.25,0.3,45,45,135],
       [0.2,0.25,0.3,45,45,180],
       [0.2,0.25,0.3,45,45,240],
       [0.2,0.25,0.3,45,45,360]]

myrobot.move_global(traj,time_step=(2,1),sim=(not wrist_test))

In [ ]:
# RELATIVE MOVEMENT
if relative_test:
    points=[[np.pi/2,0,0,0,0,0],
           [-np.pi/2,np.pi/6,0,0,0,0]]
    myrobot.move_joints_r(points,time_step=(2,1),sim=(not wrist_test))

In [ ]:
# LINE TEST

line = []
for i in range(0,50):
    line.append([0.3,0.3-i*(0.4/50),0.25,0,0,0])

myrobot.move_global(line,time_step=(2,0.1),sim=(not line_test))

In [ ]:
# FUN CIRCLE TEST

steps = 100
r = 0.075

circle = []
for t in range(steps):
    x = r * np.cos(t*2*np.pi/(steps))
    y = r * np.sin(t*2 * np.pi / (steps))
    circle.append([0.2+x,0.25+y,0.3,-45*np.sin(t*2 * np.pi / (steps)),45*np.cos(t*2 * np.pi / (steps)),0+2*(i*360/100)])

myrobot.move_global(circle,time_step=(3,0.1),sim=(not circle_test))

In [ ]:
HTML(filename="ur3e_trajectory.html")

In [ ]:
rclpy.shutdown()